# libraries

In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from collections import deque

# random seed for reproducibility
np.random.seed(42)

# Loading the Karate Club Graph
G = nx.karate_club_graph()

# layout for consistent visualization
pos = nx.spring_layout(G, seed=42)

print(f"Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.")

Graph loaded: 34 nodes, 78 edges.


# Modularity Matrix 

In [ ]:
def get_modularity_matrix(G):
    A = nx.to_numpy_array(G)
    k = A.sum(axis=1) 
    m = G.number_of_edges()
    k_k_T = np.outer(k, k)
    B = A - (k_k_T / (2 * m))
    return B

# Spectral Bisection

In [ ]:
def spectral_split(B_sub, node_list):
    evals, evecs = la.eigh(B_sub)
    
    # Sorting
    idx = np.argsort(evals)[::-1]
    evals = evals[idx]
    evecs = evecs[:, idx]
    
    largest_eigenvalue = evals[0]
    leading_eigenvector = evecs[:, 0]
    
    # Stopping Criterion: If max eigenvalue <= 0, do not split
    if largest_eigenvalue <= 1e-10:
        return None, None, False
        
    # Split based on sign of eigenvector components
    group1 = []
    group2 = []
    
    for i, val in enumerate(leading_eigenvector):
        if val > 0:
            group1.append(node_list[i])
        else:
            group2.append(node_list[i])
            
    return group1, group2, True

# Metrics (Centralities and coefficient) Calculation

In [ ]:
def compute_metrics(G_current, iteration_idx, metrics_history):
    # Degree Centrality
    deg = nx.degree_centrality(G_current)
    
    # Betweenness Centrality 
    bet = nx.betweenness_centrality(G_current)
    
    # Closeness Centrality 
    clo = nx.closeness_centrality(G_current)
    
    # Clustering Coefficient 
    clus = nx.clustering(G_current)
    
    for node in G_current.nodes():
        metrics_history['degree'][node].append((iteration_idx, deg[node]))
        metrics_history['betweenness'][node].append((iteration_idx, bet[node]))
        metrics_history['closeness'][node].append((iteration_idx, clo[node]))
        metrics_history['clustering'][node].append((iteration_idx, clus[node]))